In [ ]:
import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.linalg import SparseVector
from scipy.sparse import load_npz
import utils


os.environ['PYSPARK_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\Scripts\ipython.exe'

spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc=spark.sparkContext
spark

In [ ]:
df_id=pd.read_parquet("results/bug_pair_ids.parquet")
threshold=0.8

In [ ]:
#toy_df=pd.DataFrame([["Hi how are you John?", 100], ["Hi how are you John my bro?", 101], ["Bro, what girls?", 102]], columns=["text", "id"])

In [ ]:
#toy_df

In [ ]:
""" sparse_repr, _=utils.compute_sparse_repr(toy_df["text"])
pairs, num_of_pairs=utils.compute_cosine_similarity(sparse_repr, toy_df["id"], 0.0)
pairs """

In [ ]:
sparse_repr=load_npz("results/bug_pair_ids.npz")

In [ ]:
def csr_to_sparse_vector(row):
    return SparseVector(row.shape[1], list(zip(row.indices, row.data)))

docs_sparse_forSpark = [csr_to_sparse_vector(sparse_repr.getrow(i)) for i in range(sparse_repr.shape[0])]
doc_ids = df_id["_id"].reset_index(drop=True)

In [ ]:
docs_sparse_forSpark[1]

In [ ]:
print(sparse_repr[1].indices)
print(sparse_repr[1].data)

In [ ]:
sparse_repr[1,6]

In [ ]:
sparse_repr[1].toarray()

In [ ]:
def sparse_argsort(matrix, idx):
    row=matrix.getrow(idx)
    sorted_indices = np.argsort(row.data)[::-1]
    return row.indices[sorted_indices].tolist()
sorted_index_term_doc=[sparse_argsort(sparse_repr, idx) for idx in range(sparse_repr.shape[0])]

In [ ]:
d_star=sparse_repr.max(axis=0).toarray().reshape(-1)
#d_star=d_star/np.linalg.norm(d_star, 2)

In [ ]:
sparse_repr.toarray()

In [ ]:
sparse_repr[:,8].toarray()

In [ ]:
d_star
#{0: 0.4472, 3: 0.4472, 4: 0.4472, 5: 0.4472, 8: 0.4472})
##[3, 4, 0, 8, 5]

In [ ]:
d_star_sc=sc.broadcast(d_star)
rdd_forMap=sc.parallelize([(doc_ids[i], (docs_sparse_forSpark[i], sorted_index_term_doc[i])) for i in range(sparse_repr.shape[0])])

In [ ]:
def b_d(sparse_repr, term_order):
    sparse_repr_tmp=[sparse_repr[t] for t in term_order]
    d_star_tmp=[d_star_sc.value[t] for t in term_order]
    cum_sum=0
    index=0

    for i in range(len(d_star_tmp)):
        mult_val=sparse_repr_tmp[i]*d_star_tmp[i]
        cum_sum+=mult_val
        index=i
        if cum_sum>=threshold:
            index=index-1
            break
    print(index, term_order[index])
    return index


def my_map(elem):
    #result=[]
    doc_id=elem[0]
    sparse_repr=elem[1][0]
    sorted_index=elem[1][1]

    #bound=b_d(sparse_repr, sorted_index)
    result=[((t_idx, (doc_id, sparse_repr))) for t_idx in sorted_index]#[bound+1::]]

    """ for i, t_idx in enumerate(sorted_index):
        if i>bound:
            result.append( (t_idx, (doc_id, sparse_repr)) ) """
    return result

rdd_forReduce=rdd_forMap.flatMap(my_map)


In [ ]:
""" from itertools import chain

def flatmap(func, iterable):
    return list(chain.from_iterable(map(func, iterable)))

iter=flatmap(my_map, rdd_forMap)
iter"""

In [ ]:
docs_sparse_forSpark[0].dot(docs_sparse_forSpark[1])

In [ ]:
def max_of_intersection(list1, list2):
    max=0
    i = 0
    j = 0
    while i < len(list1) and j < len(list2):
        elem1=list1[i]
        elem2=list2[j]

        if elem1 == elem2:
            if elem1>max:
                max=elem1
            i += 1
            j += 1
        elif elem1 < elem2:
            i += 1
        else:
            j += 1

    return max


def my_reduce(elem):
    result=[]
    #For directly prune the symmetric pairs 
    pairs_dict={}
    key=elem[0]
    values=elem[1]
    
    for id1, d1 in values:
        for id2, d2 in values:
            
            #print("CIAO", (id1,id2, sim))
            #print(key, d1.indices, d2.indices)
            max_int=max_of_intersection(d1.indices, d2.indices)
            #print("KEY: ",key, " Max_inters:", max_int, "\n")
            if id1!=id2 and key==max_int and (not pairs_dict.get((id2, id1), False)):
                sim=round(d1.dot(d2), 4)
                #because vector are already normalized
                if sim>=threshold:
                    pairs_dict[(id1, id2)]=True
                    result.append((id1, id2, sim))
    print()
    return result

#result_pairs=rdd_forReduce.groupByKey().flatMap(my_reduce)


#gb=rdd_forReduce.groupByKey().mapValues(list).collect()

In [ ]:
#gb

In [ ]:
#flatmap(my_reduce, gb)

In [ ]:
risultato=rdd_forReduce.groupByKey().mapValues(list).flatMap(my_reduce).collect()
#print(risultato)

In [ ]:
risultato

In [ ]:
len(risultato)

In [ ]:
print(set(np.array([[e[0], e[1]] for e in risultato]).flatten()))

In [ ]:
spark.stop()